I had started with [Evidence](https://evidence.dev/) but evidence's experience wasn't working out for me. The query strings needed to be changed at every step to even get it working and there were basically no error messages

## 1. Structure Investigation

A grisly approximation of the shape function in SQL, may Lord have mercy

In [1]:


num_cols = _deepnote_execute_sql("""-- Number of columns 
SELECT COUNT(*) FROM `labocr-267706.eda.INFORMATION_SCHEMA.COLUMNS` WHERE table_name="road_safety"
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
num_cols

,f0_
0,67


In [2]:


num_rows = _deepnote_execute_sql("""-- Number of rows
SELECT COUNT(*) FROM `labocr-267706.eda.road_safety`
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
num_rows

,f0_
0,363243


In [3]:


col_types = _deepnote_execute_sql("""-- Count how many times each data type is present in the dataset
SELECT data_type,COUNT(*) FROM `labocr-267706.eda.INFORMATION_SCHEMA.COLUMNS` GROUP BY data_type
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
col_types

,data_type,f0_
0,STRING,4
1,INT64,6
2,FLOAT64,56
3,DATE,1


### 1.1. Structure of non-numerical features

In [5]:


non_numerical_columns = _deepnote_execute_sql("""SELECT
    STRING_AGG(column_name,",")
  FROM
    `labocr-267706.eda.INFORMATION_SCHEMA.COLUMNS`
  WHERE
    data_type IN ("STRING",
      "DATE")
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
non_numerical_columns

,f0_
0,"Accident_Index,Date,Time,Local_Authority__High..."


In [13]:
non_numerical_columns_str = non_numerical_columns["f0_"][0]

In [19]:


target_columns = _deepnote_execute_sql("""-- Okay I am using a trick with I am not sure would work anywhere else, this is jinjasql 
-- and Deepnote has an integration with SQL to execute this sweetly. 
-- You'll need a certain amount of modification to get this working else where properly
SELECT
  {{non_numerical_columns_str | sqlsafe}}
FROM
  `labocr-267706.eda.road_safety`
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
target_columns

,Accident_Index,Date,Time,Local_Authority__Highway_,LSOA_of_Accident_Location
0,2015440080946,None,None,None,None
1,2015440080946,None,None,None,None
2,2015440083952,None,None,None,None
3,2015440083952,None,None,None,None
4,2015440101052,None,None,None,None
...,...,...,...,...,...
363238,201597NE00308,2015-08-01,00:58,S12000044,None
363239,201597NE00308,2015-08-01,00:58,S12000044,None
363240,201597UC71106,2015-06-17,09:20,S12000008,None
363241,201597UC71106,2015-06-17,09:20,S12000008,None


In [21]:
target_columns.describe()
# Rewriting describe in SQL is just too much work

,Accident_Index,Date,Time,Local_Authority__Highway_,LSOA_of_Accident_Location
count,363243,319866,319822,319866,298758
unique,140056,365,1439,204,25979
top,201543P296025,2015-02-14,17:30,E10000017,E01028497
freq,1332,2144,2972,8457,1456


### 1.2. Structure of numerical features

In [22]:


numerical_columns = _deepnote_execute_sql("""SELECT
    STRING_AGG(column_name,",")
  FROM
    `labocr-267706.eda.INFORMATION_SCHEMA.COLUMNS`
  WHERE
    data_type IN ("INT64",
      "FLOAT64")
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
numerical_columns

,f0_
0,"Vehicle_Reference_df_res,Vehicle_Type,Towing_a..."


In [24]:


numericalcols_data = _deepnote_execute_sql("""SELECT
  {{numerical_columns["f0_"][0] | sqlsafe}}
FROM
  `labocr-267706.eda.road_safety`
""", 'SQL_D15DB2FC_356F_4858_A44E_CAB7271219CD')
numericalcols_data

,Vehicle_Reference_df_res,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location_Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,Hit_Object_off_Carriageway,...,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,Casualty_IMD_Decile
0,1,97.0,0.0,3.0,0.0,8.0,0.0,9.0,0.0,0.0,...,10.0,3,0.0,0.0,0.0,0.0,0.0,97,1.0,5.0
1,2,9.0,0.0,18.0,0.0,8.0,0.0,0.0,0.0,0.0,...,10.0,3,0.0,0.0,0.0,0.0,0.0,97,1.0,5.0
2,1,9.0,0.0,7.0,0.0,8.0,0.0,0.0,0.0,0.0,...,11.0,2,0.0,0.0,0.0,0.0,0.0,22,1.0,3.0
3,2,22.0,0.0,18.0,0.0,8.0,0.0,0.0,0.0,0.0,...,11.0,2,0.0,0.0,0.0,0.0,0.0,22,1.0,3.0
4,1,21.0,1.0,16.0,0.0,0.0,2.0,0.0,1.0,2.0,...,7.0,3,0.0,0.0,0.0,0.0,0.0,21,1.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363238,2,19.0,0.0,18.0,0.0,3.0,0.0,0.0,0.0,0.0,...,7.0,3,0.0,0.0,2.0,0.0,0.0,19,NaN,NaN
363239,2,19.0,0.0,18.0,0.0,3.0,0.0,0.0,0.0,0.0,...,7.0,3,0.0,0.0,0.0,0.0,0.0,19,NaN,NaN
363240,1,19.0,0.0,18.0,0.0,1.0,0.0,0.0,0.0,0.0,...,6.0,3,0.0,0.0,0.0,0.0,0.0,19,NaN,NaN
363241,2,9.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,6.0,3,0.0,0.0,0.0,0.0,0.0,19,NaN,NaN


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0250b4f5-a1fd-4aaa-8b27-2847b6649f1e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>